<a href="https://colab.research.google.com/github/greensky0107/self_study/blob/main/Day29_Classification_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

XGBoost

Tree Ensemble 중 성능이 좋은 알고리즘

eXtreme Gradient Boosting를 줄여서 XGBoost라고 함  (Gradient는 쉽게 기울기로 이야기)


약한 학습기가 계속해서 업데이트를 하며 좋은 모델을 만들어 간다.

Boosting (Ensemble) 기반의 알고리즘

Kaggle (글로벌 AI 경진대회)에서 뛰어난 성능을 보이면서 인기가 높아짐

In [1]:
#Step 1. Library 불러오기

# Pandas 불러오기
import pandas as pd

# Numpy 불러오기
import numpy as np

# Matplotlib 불러오기
import matplotlib.pyplot as plt

# Seaborn 불러오기
import seaborn as sns

# 사이킷런 내장데이터 확인
import sklearn.datasets
sklearn.datasets.__all__  # load로 된 부분이 데이터셋

# 데이터 불러오기
from sklearn.datasets import load_breast_cancer  # load_데이터 세트

# 데이터 분리
from sklearn.model_selection import train_test_split

# 모델 불러오기
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
# XGBoost
from xgboost import XGBClassifier

# 평가
# accuracy (정확도)
from sklearn.metrics import accuracy_score
# precision (정밀도)
from sklearn.metrics import precision_score
# recall (재현율)
from sklearn.metrics import recall_score
# f1 (정밀도와 재현율의 조화평균)
from sklearn.metrics import f1_score
# roc auc (Receiver Operation Chracteristic Curve,  Area Under the Curve)
from sklearn.metrics import roc_auc_score

In [29]:
#Step2 Data Split (train vs. test.  size 얼마로 할 지 정하기.  50%로 지정)

from sklearn.datasets import load_breast_cancer

def make_dataset():
    cancer = load_breast_cancer()
    df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
    df['target'] = cancer.target
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop('target', axis=1), df['target'], test_size=0.5, random_state=5000)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = make_dataset()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((284, 30), (285, 30), (284,), (285,))

In [30]:
#Step3 Target 확인
y_train.value_counts()

,count
target,
1,170
0,114


In [31]:
#Step 4 XGBoost 실행

#모델 변수 정의_ dt_clf(의사결정나무) / rf_clf(랜덤포레스트) / lr_clf(로지스틱회귀) 등
from xgboost import XGBClassifier
model = XGBClassifier(random_state=0, eval_metric='logloss')

#학습
model.fit(X_train, y_train)

#예측
pred = model.predict(X_test)

#평가
#Accuracy(정확도)
accuracy_score(y_test, pred)

#test size 50% 일 때 0.933
#test size 30% 일 때 0.964

0.9333333333333333

In [32]:
#Step 4 Evaluation

#Precision(정밀도)
precision_score(y_test, pred)

#test size 50% 일 때 0.942
#test size 30% 일 때 0.966

0.9421052631578948

In [33]:
#Step 4 Evaluation

#Recall(재현율)
recall_score(y_test, pred)

#test size 50% 일 때 0.957
#test size 30% 일 때 0.982

0.9572192513368984

In [34]:
#Step 4 Evaluation

# f1 (정밀도와 재현율의 조화평균)
f1_score(y_test, pred)

#test size 50% 일 때 0.949
#test size 30% 일 때 0.974

0.9496021220159151

In [35]:
#Step 4 Evaluation

# roc_auc
model = XGBClassifier(random_state=0, eval_metric='auc')
model.fit(X_train, y_train)
pred = model.predict_proba(X_test)
roc_auc_score(y_test, pred[:,1])
print(roc_auc_score(y_test, pred[:,1]))

#test size 50% 일 때 0.985
#test size 30% 일 때 0.992

0.9855669540543491


XGBooster Hyper Parameter 종류

booster (기본값 gbtree) : 부스팅 알고리즘 (또는 dart, gblinear)

objective (기본값 binary:logistic) : 이진분류 (다중분류: multi:softmax)

max_depth (기본값 6) : 최대 한도 깊이

learning_rate (기본값 0.1) : 학습률

n_estimators (기본값 100) : 트리의 수

subsample (기본값 1) : 훈련 샘플 개수의 비율

colsample_bytree (기본값 1) : 특성 개수의 비율

n_jobs (기본값 1) : 사용 코어 수 (-1: 모든 코어를 다 사용)

In [36]:
#Step 5 XGBooster Hyperparameter 조정 (max depth, minimum sample leaf, minimum sample split 조정)

model = XGBClassifier(random_state=0, eval_metric='logloss',
                      booster = 'gbtree',
                      objective = 'binary:logistic',
                      max_depth = 5,
                      learning_rate = 0.05,
                      n_estimators = 500,
                      subsample = 1,
                      colsample_bytree = 1,
                      n_jobs = -1


# - booster(기본값 gbtree): 부스팅 알고리즘 (또는 dart, gblinear)
# - objective(기본값 binary:logistic): 이진분류 (다중분류: multi:softmax)
# - max_depth(기본값 6): 최대 한도 깊이
# - learning_rate(기본값 0.1): 학습률
# - n_estimators(기본값 100): 트리의 수
# - subsample(기본값 1): 훈련 샘플 개수의 비율
# - colsample_bytree(기본값 1): 특성 개수의 비율
# - n_jobs(기본값 1): 사용 코어 수 (-1: 모든 코어를 다 사용)
                     )
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9403508771929825

In [37]:
# Early Stopping

model = XGBClassifier(random_state=0, eval_metric='logloss',
                     learning_rate = 0.05,
                      n_estimators = 500,
                      early_stopping_rounds=10) # Moved early_stopping_rounds here
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set)  # Removed early_stopping_rounds from here
pred = model.predict(X_test)
accuracy_score(y_test, pred)

[0]	validation_0-logloss:0.61671
[1]	validation_0-logloss:0.58500
[2]	validation_0-logloss:0.55609
[3]	validation_0-logloss:0.53010
[4]	validation_0-logloss:0.50590
[5]	validation_0-logloss:0.48386
[6]	validation_0-logloss:0.46380
[7]	validation_0-logloss:0.44550
[8]	validation_0-logloss:0.42830
[9]	validation_0-logloss:0.41250
[10]	validation_0-logloss:0.39817
[11]	validation_0-logloss:0.38463
[12]	validation_0-logloss:0.37075
[13]	validation_0-logloss:0.35900
[14]	validation_0-logloss:0.34712
[15]	validation_0-logloss:0.33639
[16]	validation_0-logloss:0.32631
[17]	validation_0-logloss:0.31648
[18]	validation_0-logloss:0.30781
[19]	validation_0-logloss:0.29918
[20]	validation_0-logloss:0.29225
[21]	validation_0-logloss:0.28449
[22]	validation_0-logloss:0.27696
[23]	validation_0-logloss:0.27044
[24]	validation_0-logloss:0.26435
[25]	validation_0-logloss:0.25887
[26]	validation_0-logloss:0.25368
[27]	validation_0-logloss:0.24854
[28]	validation_0-logloss:0.24394
[29]	validation_0-loglos

0.9333333333333333